In [ ]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
!echo "Downloading files..."
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/holdout.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/mini_holdout.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/mini_holdout_answers.csv

!echo "Unzipping files..."
!unzip -q /content/training1.zip
!unzip -q /content/training2.zip
!unzip -q /content/holdout.zip
!unzip -q /content/mini_holdout.zip

# Combine the two traning directories
!echo "Merging training data..."
!mkdir /content/training
!mv /content/training1/* /content/training
!mv /content/training2/* /content/training

# Cleanup
!echo "Cleaning up..."
!rmdir /content/training1
!rmdir /content/training2
!rm training1.zip
!rm training2.zip
!rm holdout.zip
!rm mini_holdout.zip

!echo "Data ready."

In [ ]:
# Import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# We're using keras' ImageDataGenerator class to load our image data.
# See (https://keras.io/api/preprocessing/image/#imagedatagenerator-class) for details
#
# A couple of things to note:
# 1. We're specifying a number for the seed, so we'll always get the same shuffle and split of our images.
# 2. Class names are inferred automatically from the image subdirectory names.
# 3. We're splitting the training data into 80% training, 20% validation.


training_dir = '/content/training/'
image_size = (100, 100)

# Split up the training data images into training and validations sets
# We'll use and ImageDataGenerator to do the splits
# ImageDataGenerator can also be used to do preprocessing and agumentation on the files as can be seen with rescale

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=.2
        )
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=.2
        )

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size = image_size,
        subset="training",
        batch_size=32,
        class_mode='sparse',
        seed=42,shuffle=True)
validation_generator = validation_datagen.flow_from_directory(
        training_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='sparse',
        subset="validation",
        seed=42)



In [ ]:
#these might come in handy
target_names = ['Speed_20', 'Speed_30', 'Speed_50', 'Speed_60', 'Speed_70',
               'Speed_80','Speed_Limit_Ends', 'Speed_100', 'Speed_120', 'Overtaking_Prohibited',
               'Overtakeing_Prohibited_Trucks', 'Priority', 'Priority_Road_Ahead', 'Yield', 'STOP',
               'Entry_Forbidden', 'Trucks_Forbidden', 'No_Entry(one-way traffic)', 'General Danger(!)', 'Left_Curve_Ahead',
               'Right_Curve_Ahead', 'Double_Curve', 'Poor_Surface_Ahead', 'Slippery_Surface_Ahead', 'Road_Narrows_On_Right',
               'Roadwork_Ahead', 'Traffic_Light_Ahead', 'Warning_Pedestrians', 'Warning_Children', 'Warning_Bikes',
               'Ice_Snow', 'Deer_Crossing', 'End_Previous_Limitation', 'Turning_Right_Compulsory', 'Turning_Left_Compulsory',
               'Ahead_Only', 'Straight_Or_Right_Mandatory', 'Straight_Or_Left_Mandatory', 'Passing_Right_Compulsory', 'Passing_Left_Compulsory',
               'Roundabout', 'End_Overtaking_Prohibition', 'End_Overtaking_Prohibition_Trucks']

In [ ]:
# View 9 images and their class labels
plt.figure(figsize=(10, 10))
images, labels = next(train_generator)  # Assuming train_generator is a generator
batch_size = images.shape[0]

for i in range(min(9, batch_size)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow((images[i] * 255).astype("uint8"))
    plt.title(int(labels[i]))
    plt.axis("off")

plt.show()

In [ ]:
# VGG19 Architecture for Road Sign Classification
# Both custom implementation and transfer learning approaches

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

def create_custom_vgg19(input_shape=(100, 100, 3), num_classes=43):
    """
    Custom VGG19 implementation optimized for road signs
    """
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.0005)),
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        # Block 4
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        # Block 5
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        BatchNormalization(),
        Dropout(0.5),

        # Classifier (Modified for better performance)
        GlobalAveragePooling2D(),  # Better than Flatten for generalization
        Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.6),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

def create_vgg19_transfer_learning(input_shape=(100, 100, 3), num_classes=43, trainable_layers=5):
    """
    VGG19 with transfer learning - RECOMMENDED approach
    """
    # Load pre-trained VGG19 (trained on ImageNet)
    base_model = VGG19(
        weights='imagenet',
        include_top=False,  # Don't include final classification layer
        input_shape=input_shape
    )

    # Freeze early layers, train only the last few
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.6),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])

    return model

def create_vgg19_fine_tuned(input_shape=(100, 100, 3), num_classes=43):
    """
    VGG19 with full fine-tuning - For maximum performance
    """
    base_model = VGG19(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Initially freeze all layers
    base_model.trainable = False

    # Add custom head
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.6)(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    return model, base_model

# Choose your approach (recommended: transfer learning)
print("Creating VGG19 models...")

# Option 1: Custom VGG19 (train from scratch)
print("1. Custom VGG19 (from scratch):")
custom_vgg19 = create_custom_vgg19()
print(f"Parameters: {custom_vgg19.count_params():,}")

# Option 2: Transfer Learning VGG19 (RECOMMENDED)
print("\n2. VGG19 Transfer Learning (RECOMMENDED):")
transfer_vgg19 = create_vgg19_transfer_learning()
print(f"Parameters: {transfer_vgg19.count_params():,}")

# Option 3: Fine-tuned VGG19 (for maximum performance)
print("\n3. VGG19 Fine-tuned:")
finetuned_vgg19, base_model = create_vgg19_fine_tuned()
print(f"Parameters: {finetuned_vgg19.count_params():,}")

# Let's use the transfer learning model (most practical)
model = transfer_vgg19

# Compile with different learning rates for transfer learning
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower LR for transfer learning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Enhanced data preprocessing for VGG19
train_datagen_vgg = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=False,
    fill_mode='nearest',
    validation_split=0.2
)

validation_datagen_vgg = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Create data generators
train_generator_vgg = train_datagen_vgg.flow_from_directory(
    training_dir,
    target_size=image_size,
    subset="training",
    batch_size=32,  # VGG19 can handle larger batches
    class_mode='sparse',
    seed=42,
    shuffle=True
)

validation_generator_vgg = validation_datagen_vgg.flow_from_directory(
    training_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='sparse',
    subset="validation",
    seed=42,
    shuffle=False
)

# Calculate class weights
class_weights = {}
unique_classes, class_counts = np.unique(train_generator_vgg.classes, return_counts=True)
total_samples = len(train_generator_vgg.classes)

for i, count in enumerate(class_counts):
    class_weights[i] = total_samples / (len(unique_classes) * count)

# Callbacks for VGG19 training
callbacks_vgg = [
    EarlyStopping(
        monitor='val_loss',
        patience=12,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.0001
    ),

    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-8,
        verbose=1
    ),

    ModelCheckpoint(
        'best_vgg19_road_sign_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Training strategy for transfer learning
print("\n" + "="*60)
print("TRAINING STRATEGY: TWO-PHASE APPROACH")
print("="*60)
print("Phase 1: Train only the custom head (frozen base)")
print("Phase 2: Fine-tune the entire network (unfrozen base)")
print("="*60)

# PHASE 1: Train with frozen base model
print("\nPHASE 1: Training custom head only...")
history_phase1 = model.fit(
    train_generator_vgg,
    epochs=20,
    validation_data=validation_generator_vgg,
    callbacks=callbacks_vgg,
    class_weight=class_weights,
    verbose=1
)

# PHASE 2: Fine-tune the entire model
print("\nPHASE 2: Fine-tuning entire network...")

# Unfreeze the base model for fine-tuning
if hasattr(model.layers[0], 'trainable'):
    model.layers[0].trainable = True

# Use a much lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.00001),  # Very low LR for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with fine-tuning
history_phase2 = model.fit(
    train_generator_vgg,
    epochs=30,
    validation_data=validation_generator_vgg,
    callbacks=callbacks_vgg,
    class_weight=class_weights,
    verbose=1
)

# Combine training histories
def combine_histories(hist1, hist2):
    """Combine two training histories"""
    combined = {}
    for key in hist1.history.keys():
        combined[key] = hist1.history[key] + hist2.history[key]
    return combined

combined_history = combine_histories(history_phase1, history_phase2)

# Comprehensive evaluation function (same as before)
def comprehensive_evaluation_vgg(model, validation_generator, target_names):
    """Comprehensive evaluation for VGG19 model"""
    validation_generator.reset()
    predictions = model.predict(validation_generator, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes

    accuracy = np.mean(predicted_classes == true_classes)
    f1_macro = f1_score(true_classes, predicted_classes, average='macro')
    f1_weighted = f1_score(true_classes, predicted_classes, average='weighted')
    precision_macro = precision_score(true_classes, predicted_classes, average='macro')
    precision_weighted = precision_score(true_classes, predicted_classes, average='weighted')
    recall_macro = recall_score(true_classes, predicted_classes, average='macro')
    recall_weighted = recall_score(true_classes, predicted_classes, average='weighted')

    print("=" * 60)
    print("VGG19 PERFORMANCE METRICS")
    print("=" * 60)
    print(f"Accuracy:           {accuracy:.4f}")
    print(f"F1-Score (Macro):   {f1_macro:.4f}")
    print(f"F1-Score (Weighted): {f1_weighted:.4f}")
    print(f"Precision (Macro):  {precision_macro:.4f}")
    print(f"Precision (Weighted): {precision_weighted:.4f}")
    print(f"Recall (Macro):     {recall_macro:.4f}")
    print(f"Recall (Weighted):  {recall_weighted:.4f}")
    print("=" * 60)

    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted
    }

# Plot training history for both phases
def plot_vgg19_training_history(combined_history, phase1_epochs=20):
    """Plot training history showing both phases"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    epochs = range(1, len(combined_history['accuracy']) + 1)

    # Accuracy plot
    ax1.plot(epochs, combined_history['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
    ax1.plot(epochs, combined_history['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
    ax1.axvline(x=phase1_epochs, color='gray', linestyle='--', alpha=0.7, label='Phase 1 → Phase 2')
    ax1.set_title('VGG19 Model Accuracy (Two-Phase Training)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Loss plot
    ax2.plot(epochs, combined_history['loss'], 'b-', label='Training Loss', linewidth=2)
    ax2.plot(epochs, combined_history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    ax2.axvline(x=phase1_epochs, color='gray', linestyle='--', alpha=0.7, label='Phase 1 → Phase 2')
    ax2.set_title('VGG19 Model Loss (Two-Phase Training)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Evaluate the final model
print("\nFinal evaluation of VGG19 model:")
vgg_results = comprehensive_evaluation_vgg(model, validation_generator_vgg, target_names)

# Plot training history
plot_vgg19_training_history(combined_history)

# Save the final model
model.save('final_vgg19_road_sign_model.h5')

print("\n" + "="*60)
print("VGG19 IMPLEMENTATION SUMMARY")
print("="*60)
print("✓ Transfer learning from ImageNet weights")
print("✓ Two-phase training (frozen → fine-tuned)")
print("✓ Custom classification head optimized for road signs")
print("✓ Class weights for balanced learning")
print("✓ Advanced data augmentation")
print("✓ Comprehensive metric evaluation")
print("✓ Model saved as 'final_vgg19_road_sign_model.h5'")
print("="*60)

print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 98, 98, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 98, 98, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 96, 96, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 46, 46, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 44, 44, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 20, 20, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 18, 18, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 7, 7, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 5, 5, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,887,851 (22.46 MB)

 Trainable params: 5,882,859 (22.44 MB)

...

1961/1961 ━━━━━━━━━━━━━━━━━━━━ 150s 77ms/step - accuracy: 0.4060 - loss: 3.1982 - val_accuracy: 0.6583 - val_loss: 2.4509 - learning_rate: 5.0000e-04
Epoch 5/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5892 - loss: 2.7716
Epoch 5: val_accuracy improved from 0.65833 to 0.70641, saving model to best_optimized_road_sign_model.h5


1961/1961 ━━━━━━━━━━━━━━━━━━━━ 149s 76ms/step - accuracy: 0.5892 - loss: 2.7715 - val_accuracy: 0.7064 - val_loss: 2.4905 - learning_rate: 5.0000e-04
Epoch 6/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.7157 - loss: 2.5044
Epoch 6: val_accuracy improved from 0.70641 to 0.90792, saving model to best_optimized_road_sign_model.h5


1961/1961 ━━━━━━━━━━━━━━━━━━━━ 147s 75ms/step - accuracy: 0.7157 - loss: 2.5044 - val_accuracy: 0.9079 - val_loss: 2.3688 - learning_rate: 5.0000e-04
Epoch 7/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7956 - loss: 2.3106
Epoch 7: val_accuracy improved from 0.90792 to 0.93866, saving model to best_optimized_road_sign_model.h5


1961/1961 ━━━━━━━━━━━━━━━━━━━━ 152s 78ms/step - accuracy: 0.7957 - loss: 2.3106 - val_accuracy: 0.9387 - val_loss: 1.8658 - learning_rate: 5.0000e-04
Epoch 8/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8374 - loss: 2.1557
Epoch 8: val_accuracy improved from 0.93866 to 0.95791, saving model to best_optimized_road_sign_model.h5


1961/1961 ━━━━━━━━━━━━━━━━━━━━ 150s 77ms/step - accuracy: 0.8374 - loss: 2.1557 - val_accuracy: 0.9579 - val_loss: 1.7440 - learning_rate: 5.0000e-04
Epoch 9/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8544 - loss: 2.0848
Epoch 9: val_accuracy did not improve from 0.95791
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 150s 76ms/step - accuracy: 0.8544 - loss: 2.0848 - val_accuracy: 0.9327 - val_loss: 1.8078 - learning_rate: 5.0000e-04
Epoch 10/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8607 - loss: 2.0397
Epoch 10: val_accuracy did not improve from 0.95791
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 146s 74ms/step - accuracy: 0.8607 - loss: 2.0397 - val_accuracy: 0.9398 - val_loss: 1.7661 - learning_rate: 5.0000e-04
Epoch 11/100
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.8800 - loss: 1.9271
Epoch 11: val_accuracy did not improve from 0.95791
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 148s 75ms/step - accuracy: 0.8800 - loss: 1.9271 - val_accuracy: 0.9547 - val_loss: 1.7040 -

## Testing the model
Once you have built and trained your model, the next step is to run the mini holdout images through it and see how well your model does at making predictions for images it has never seen before.

Since loading these images and formatting them for the model can be tricky, you may find the following code useful. This code only uses your model to predict the class label for a given image. You'll still need to compare those predictions to the "ground truth" class labels in `mini_holdout_answers.csv` to evaluate how well the model does.

Previously, you were given a file that would check your results. This time you're given the answers to the first mini holdout dataset. You'll need to compare those predictions against the "ground truth" class labels in `mini_holdout_answers.csv` to evaluate how well the model does.

Make sure to use the insights gained from the mini hold out dataset in your executive summary.


```
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_dir = '/content/'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        classes=['mini_holdout'],
        target_size=image_size,
        class_mode='sparse',
        shuffle=False)
probabilities = model.predict(test_generator)
predictions = [np.argmax(probas) for probas in probabilities]
```



##Mini Hold out Dataset


Once you feel confident, you will need to predict for the full holdout dataset using the following code, and submit your csv file:

```
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_dir = '/content/'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        classes=['holdout'],
        target_size=image_size,
        class_mode='sparse',
        shuffle=False)
probabilities = model.predict(test_generator)
predictions = [np.argmax(probas) for probas in probabilities]
```